# AML Project - Anomaly Segmentation

## Notebook setup

In [1]:
%cd /content
!rm -r AnomalySegmentation_CourseProjectBaseCode
!git clone https://github.com/AML-project-AnomalySegmentation/AnomalySegmentation_CourseProjectBaseCode.git


/content
rm: cannot remove 'AnomalySegmentation_CourseProjectBaseCode': No such file or directory
Cloning into 'AnomalySegmentation_CourseProjectBaseCode'...
remote: Enumerating objects: 21917, done.
remote: Counting objects: 100% (2716/2716), done.
remote: Compressing objects: 100% (2702/2702), done.
remote: Total 21917 (delta 15), reused 2714 (delta 14), pack-reused 19201
Receiving objects: 100% (21917/21917), 11.97 GiB | 43.65 MiB/s, done.
Resolving deltas: 100% (3809/3809), done.
Updating files: 100% (27414/27414), done.


In [13]:
!pip3 install visdom
!pip3 install ood_metrics
!pip3 install cityscapesScripts


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.9 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=ebefabecb9a002a2a4db8a959c69fef7d2bfeee238c54494478201922f88a5fa
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


## Steps

In [1]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/eval
!python converter.py

/content/AnomalySegmentation_CourseProjectBaseCode/eval
../dataset/Cityscapes
Processing 5000 annotation files
Progress: 5.32 % 

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

Progress: 100.0 % 

### Step 2A

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
DISCRIMINANTS = ["msp", "maxlogit", "maxentropy"]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  for disc in (DISCRIMINANTS):
    print(f"Dataset: {d} Discriminant: {disc}")
    !python evalAnomaly.py  --input {path} --discriminant {disc}

In [ ]:
DISCRIMINANTS = ["msp", "maxlogit", "maxentropy"]
for d in DISCRIMINANTS:
   !python eval_iou.py --discriminant {d}

In [ ]:
TEMPERATURE = [0.5, 0.75, 1.1]
for d in TEMPERATURE:
   !python eval_iou.py --temperature {d}

In [6]:
!python optimalTemperature.py

Loading model: ../trained_models/erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
/
ROOT: ../dataset/Cityscapes
IMAGES: ../dataset/Cityscapes/leftImg8bit/val
LABELS: ../dataset/Cityscapes/gtFine/val
../dataset/Cityscapes/leftImg8bit/val ../dataset/Cityscapes/gtFine/val
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Before temperature - NLL: 11.816, ECE: 0.000
Optimal temperature: 10.495
After temperature - NLL: 3.248, ECE: 0.000
Done!


### Step 2B

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
TEMPERATURES = [1,0.5,0.75,1.1]
for d in (DATASETS):
  path = f"'Validation_Dataset/{d}/images/*.*'"
  for t in (TEMPERATURES):
    print(f"Dataset: {d} Temeperature: {t}")
    !python evalAnomaly.py  --input {path} --temperature {t}


## Step 3

### Train Bisenet

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder --model 'bisenetv1'

### Train ENet

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder --model 'enet'

### Train Erfnet

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder --model 'erfnet'

In [ ]:
MODELS = ["ErfNet", "ENet", "BiseNet"]
for m in MODELS:
  !python void_eval_iou.py --model {m}

In [ ]:
DATASETS = ["RoadAnomaly21", "RoadObsticle21", "FS_LostFound_full", "fs_static", "RoadAnomaly"]
MODELS = ["ErfNet", "ENet", "BiseNet"]
for m in MODELS:
  for d in (DATASETS):
    path = f"'Validation_Dataset/{d}/images/*.*'"
    print(f"Dataset: {d}")
    !python VoidEvalAnomaly.py  --input {path} --model {m}

In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main_training_void.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder



## Step 4



## Train erfNet


In [ ]:
%cd /content/AnomalySegmentation_CourseProjectBaseCode/train
!python main.py --savedir "/trainingdata" --resume --epochs-save 1 --num-epochs 3 --decoder